In [106]:
# randomly select sample
import util
import numpy as np

home_dir = '1114-2330'
filelist = util.get_filelist(home_dir=home_dir,ext='.png')
print "filelist size:", len(filelist)

size_test = 2000

data = np.load(home_dir+'/data.npz')
tt_feat = data['tt_feat']
tt_handles = data['tt_handles']
tt_pos = data['tt_pos']
print tt_feat.shape,tt_pos.shape

inds = np.random.choice(a=len(tt_feat),size=size_test)
print inds
inds_test = inds[:int(size_test*0.66)] 
inds_train = inds[int(size_test*0.66):]
print inds_test.shape, inds_train.shape

X_train = tt_feat[inds_train]
y_train = tt_pos[inds_train]
handles_for_train = tt_handles[inds_train]

X_test = tt_feat[inds_test]
y_test = tt_pos[inds_test]

# train
from rfdic import *
rt = random_trees(tt_feat=X_train, tt_pos=y_train, tt_handles=handles_for_train, 
                  num_class=10, num_trees=3)

from vfdic import *
vfdic = vfdic(tt_feat=X_train, tt_pos=y_train)

hvf = hidden_vfdic(tt_feat=X_train, tt_pos=y_train, tt_handles=handles_for_train, 
                  num_class=50, num_trees=20)

n_test = 4
act = np.zeros((n_test,y_test.shape[1]))
act_delta = np.zeros(4)
# test different regression methods
for i in range(len(X_test)/2):
    act[0] = rt.feat_to_act(feat=X_test[i,:])
    act[1] = rt.feat_to_pos(feat=X_test[i,:])
    act[2] = vfdic.feat_to_pos(feat=X_test[i,:])
    act[3] = hvf.predict(feat=X_test[i,:])
    
    for j in range(len(act)):
        act_delta[j] = act_delta[j] + np.sum((act[j]-y_test[i,:])**2)
    
act_delta = act_delta/i
print act_delta

filelist size: 9931
(9831, 1640) (9831, 6)
[3381 5908 2689 ..., 6115 4520 7969]
(1320,) (680,)
actural leaf amount 184.0
leaf_amount 184
actual_id length 184
max leaf_id 183
H shape (680, 184)
1020
(1020, 1640) (1020, 6)
[ 0.31771067  0.32568538  0.10119205  0.06074151]


In [125]:
import cv2
import wrinkle2
import numpy as np
import matplotlib.pyplot as plt

class hidden_vfdic():
    def __init__(self,tt_handles,tt_pos,tt_feat,num_class=100, num_trees=1):
        self.tt_handles = tt_handles
        self.tt_pos = tt_pos
        self.tt_feat = tt_feat
        # build the initial classifier
        # cluster the pos position:
        # give labels
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=num_class, random_state=0).fit(tt_handles)
        self.tt_hidden = kmeans.labels_

        from sklearn.ensemble import RandomForestClassifier
        # using labels to construct the random trees
        self.clf = RandomForestClassifier(n_estimators=num_trees)
        # n_estimators : integer, optional (default=10) The number of trees in the forest.
        self.clf.fit(X=tt_feat, y=self.tt_hidden)

    def predict(self,feat,alpha=1e-5):
        hidden = self.clf.predict(X=feat.reshape(1,-1))
        inds = self.tt_hidden==hidden
        X = self.tt_feat[inds]
        y = self.tt_pos[inds]
        # regression
        # begin training
        from sklearn import linear_model
        #model = linear_model.Lasso(alpha = alpha)
        model = linear_model.LinearRegression()
        model.fit(X, y)
        return model.predict(feat.reshape(1,-1))
    
for num_class in [2,5,10,20,50,100,200,500]:
    hvf = hidden_vfdic(tt_feat=X_train, tt_pos=y_train, tt_handles=handles_for_train, 
                      num_class=num_class, num_trees=20)
    ind = 200
    loss = .0
    for ind in range(len(X_test)/10):
        loss = loss + np.sum((tt_pos[ind,:]-hvf.predict(feat=tt_feat[ind,:]))**2)
    print num_class, loss 

    
# 2 5.86393553377
# 5 6.57137077746
# 10 5.9642162748
# 20 12.0250217023
# 50 6.83634713385
# 100 13.0841750396
# 200 11.9108020106
# 500 18.4366960171

2 27.8935586826
5 23.4811524887
10 8.45976237965
20 11.1325053271
50 7.51070760764
100 10.9258590931
200 9.0411196108
500 34.9849194076
